# Simulación de flujo en Colmena

In [2]:
# Vamos a construir una base de datos aleatoria con productos que tengan la variable de 
# precio, peso y dimensiones
import pandas as pd
import plotly.graph_objects as go
from IPython.display import display
import numpy as np
from AlgoritmosAuxiliares import simularDemanda
from Simulacion import Simulacion


# Variables de control 

1. (INT) Tasa de clientes que están utilizando la aplicación 
tasa_clientes_compran

2. (INT) Parámetros para el descuento 
forma_a 
forma_b 

3. (Dict) Diccionario que indica la cantidad promedio de consumo por producto
cantidad_promedio

4. (Dict) Diccionario que indica la tasa de clientes que demandan por producto
tasas

5. (INT) Tasa de clientes a la que puedes llegar a través de publicidad en un intervalo de tiempo (1 minuto, por ejemplo)
tasa_clientes_compran_nuevos 

6. (Dict) Diccionario que indica la tasa con la que nuevos clientes demandan por producto
tasas_nuevos_clientes 

7. (INT) Tiempo de simulación (en minutos)
tiempo 

8. (INT) Tasa de clientes a la que se podrá alcanzar por medio de publicidad en el intervalo de tiempo (variable de arriba)
tasa_tota_nuevos_clientes 

9. (Dict) Límite inferior del precio del producto 
limites_inferiores 

### Todas las variables que contienen la etiqueta # TODO: Estimar tienen que ser estimadas por medio de métodos frecuentistas o bayesianos una vez que se cuente con datos. 


In [3]:
# Parámetros globales 

# demanda_clientes establece el número de clientes que van a demandar un producto, más no la cantidad (esa se define posteriormente)
productos = pd.DataFrame(columns=['Id', 'Nombre', 'precio', 'peso', 'dimensiones', 'cantidad', 'demanda_clientes', 'productos_solicitados'])
nombres = ['Frijoles', 'Leche', 'Cereal', 'Arroz', 'Fruta', 'Jugos']

for i in range(len(nombres)):
    productos.loc[i] = [i, nombres[i], np.random.randint(10, 30), np.random.randint(5, 20), np.random.randint(1, 10), np.random.randint(750, 1000), 0, 0]

# En promedio, 100 clientes están solicitando productos para un camión dado
tasa_clientes_compran = 150   # TODO: Estimar

# Configuración de la tasa de descuentos 
# La tasa de descuentos está dada por forma_a / (forma_a + forma_b), de esta manera, 3 / (3+6) = 0.3 => 30% de los clientes que compran tienen un descuento
forma_a = 3 
forma_b = 6

# Arreglo que estima la cantidad promedio que se compra por producto # TODO: Estimar
cantidad_promedio = {'Frijoles': 3, 
                    'Leche': 2,
                    'Cereal': 2,
                    'Arroz': 2,
                    'Fruta': 2,
                    'Jugos': 2}

# Ahora configuramos la tasa de demanda para cada producto # TODO: Estimar
tasas = {
                    'Frijoles': 120,
                    'Leche': 115,
                    'Cereal': 70,
                    'Arroz': 98,
                    'Fruta': 45,
                    'Jugos': 28
}

# Configuramos las tasas de *NUEVOS* clientes que compran por producto 
tasa_clientes_compran_nuevos = 2 # TODO: Estimar

# Tasas de nuevos clientes que compran el producto en un intervalo de tiempo (minuto) # TODO: Estimar
tasas_nuevos_clientes = {
                    'Frijoles': 1,
                    'Leche': 2,
                    'Cereal': 0,
                    'Arroz': 1,
                    'Fruta': 0,
                    'Jugos': 0
}

i = 0
for producto in productos["Nombre"]:
    productos.loc[i, 'demanda_clientes'] = simularDemanda(tasas[producto])
    i += 1

#  Suponemos que el tiempo de llegada del camión es de 85 minutos 
tiempo = 85 # TODO: Estimar

# Esta variable representa el número de clientes que se pueden alcanzar (en una proximidad de los clientes que ya compraron) y convencerlos de comprar a través de publicidad y adds dirigidos
tasa_tota_nuevos_clientes = 30 # TODO: Estimar

# Establecemos un limite inferior del precio para cada producto (que es el precio del producto menos el 10%)
limite_inferior = {}
for producto, precio in zip(productos['Nombre'], productos['precio']):
    limite_inferior[producto] = precio - (precio * 0.1)

productos['limite_inferior'] = limite_inferior.values()

query = {
    "tasa_clientes_compran": tasa_clientes_compran,
    "forma_a": forma_a,
    "forma_b": forma_b,
    "cantidad_promedio": cantidad_promedio,
    "tasa_clientes_compran_nuevos": tasa_clientes_compran_nuevos,
    "tasas_nuevos_clientes": tasas_nuevos_clientes,
    "tiempo": tiempo,
    "tasa_tota_nuevos_clientes": tasa_tota_nuevos_clientes,
    "limites_inferiores": limite_inferior, 
    "productos": productos
}

# Para visualización de los datos 

# Vamos a construir un dataframe que contenga Nombre, cantidad_promedio, tasas_nuevos_cliente, demanda_clientes (de productos) 
productos['cantidad_promedio'] = cantidad_promedio.values()
productos['tasas_nuevos_clientes'] = tasas_nuevos_clientes.values()
productos['demanda_clientes'] = productos['demanda_clientes']
productos['limite_inferior'] = productos['limite_inferior']


display(productos)

# Simulación de Colmena 

# Creamos un objeto de tipo simulación
simulacion = Simulacion(query)
# Ejecutamos la simulación
simulacion.run(tiempo_bool = True, tiempo_cero = True)

,Id,Nombre,precio,peso,dimensiones,cantidad,demanda_clientes,productos_solicitados,limite_inferior,cantidad_promedio,tasas_nuevos_clientes
0,0,Frijoles,12,5,4,846,104,0,10.8,3,1
1,1,Leche,15,12,1,848,101,0,13.5,2,2
2,2,Cereal,19,11,5,994,81,0,17.1,2,0
3,3,Arroz,27,12,3,906,97,0,24.3,2,1
4,4,Fruta,20,7,4,860,35,0,18.0,2,0
5,5,Jugos,20,12,7,884,32,0,18.0,2,0


Métodos de descuento aplicados: {'Sorteo': 35, 'Trivia': 39, 'Cupón': 50, 'VideoJuego': 34}


Ingresos totales: 7697.688077307665 y por producto: {'Frijoles': 1555.7071169361507, 'Leche': 1353.7223247664695, 'Cereal': 1340.2881102830574, 'Arroz': 2267.3243017623004, 'Fruta': 612.0409509689363, 'Jugos': 568.6052725907502}


Espacio total: 1765 y peso total: 4651


,Id,Nombre,precio,peso,dimensiones,cantidad,demanda_clientes,productos_solicitados,limite_inferior,cantidad_promedio,tasas_nuevos_clientes
0,0,Frijoles,12,5,4,846,104,151,10.8,3,1
1,1,Leche,15,12,1,848,101,101,13.5,2,2
2,2,Cereal,19,11,5,994,81,81,17.1,2,0
3,3,Arroz,27,12,3,906,97,97,24.3,2,1
4,4,Fruta,20,7,4,860,35,35,18.0,2,0
5,5,Jugos,20,12,7,884,32,32,18.0,2,0


,Id,Nombre,precio,peso,dimensiones,cantidad,demanda_clientes,productos_solicitados,limite_inferior,cantidad_promedio,tasas_nuevos_clientes
0,0,Frijoles,11.109961,5,4,846,104,235,10.8,3,1
1,1,Leche,13.462147,12,1,848,101,166,13.5,2,2
2,2,Cereal,19.000000,11,5,994,81,81,17.1,2,0
3,3,Arroz,24.698492,12,3,906,97,151,24.3,2,1
4,4,Fruta,20.000000,7,4,860,35,35,18.0,2,0
5,5,Jugos,20.000000,12,7,884,32,32,18.0,2,0


# Simulación para el 15 de Septiembre

Las cantidades y tasas las estimamos de forma subjetiva a priori. 

In [4]:
# Parámetros globales 

# demanda_clientes establece el número de clientes que van a demandar un producto, más no la cantidad (esa se define posteriormente)
productos = pd.DataFrame(columns=['Id', 'Nombre', 'precio', 'peso', 'dimensiones', 'cantidad', 'demanda_clientes', 'productos_solicitados'])
nombres = ['Frijoles', 'Leche', 'Cereal', 'Arroz', 'Fruta', 'Jugos']

for i in range(len(nombres)):
    productos.loc[i] = [i, nombres[i], np.random.randint(10, 30), np.random.randint(5, 20), np.random.randint(1, 10), np.random.randint(2500, 5000), 0, 0]

# En promedio, 100 clientes están solicitando productos para un camión dado
tasa_clientes_compran = 150  
# Configuración de la tasa de descuentos 
# La tasa de descuentos está dada por forma_a / (forma_a + forma_b), de esta manera, 3 / (3+6) = 0.3 => 30% de los clientes que compran tienen un descuento
forma_a = 3 
forma_b = 6

# Arreglo que estima la cantidad promedio que se compra por producto 
cantidad_promedio = {'Frijoles': 6, 
                    'Leche': 3,
                    'Cereal': 2,
                    'Arroz': 6,
                    'Fruta': 4,
                    'Jugos': 5}

# Ahora configuramos la tasa de demanda para cada producto 
tasas = {
                    'Frijoles': 130,
                    'Leche': 125,
                    'Cereal': 70,
                    'Arroz': 135,
                    'Fruta': 68,
                    'Jugos': 110
}

# Configuramos las tasas de *NUEVOS* clientes que compran por producto
tasa_clientes_compran_nuevos = 2

tasas_nuevos_clientes = {
                    'Frijoles': 4,
                    'Leche': 1,
                    'Cereal': 1,
                    'Arroz': 5,
                    'Fruta': 1,
                    'Jugos': 3
}

i = 0
for producto in productos["Nombre"]:
    productos.loc[i, 'demanda_clientes'] = simularDemanda(tasas[producto])
    i += 1

#  Suponemos que el tiempo de llegada del camión es de 85 minutos
tiempo = 85 

# Esta variable representa el número de clientes que se pueden alcanzar (en una proximidad de los clientes que ya compraron) y convencerlos de comprar a través de publicidad y adds dirigidos
tasa_tota_nuevos_clientes = 60

# Establecemos un limite inferior del precio para cada producto (que es el precio del producto menos el 10%)
limite_inferior = {}
for producto, precio in zip(productos['Nombre'], productos['precio']):
    limite_inferior[producto] = precio - (precio * 0.1)

productos['limite_inferior'] = limite_inferior.values()

query = {
    "tasa_clientes_compran": tasa_clientes_compran,
    "forma_a": forma_a,
    "forma_b": forma_b,
    "cantidad_promedio": cantidad_promedio,
    "tasa_clientes_compran_nuevos": tasa_clientes_compran_nuevos,
    "tasas_nuevos_clientes": tasas_nuevos_clientes,
    "tiempo": tiempo,
    "tasa_tota_nuevos_clientes": tasa_tota_nuevos_clientes,
    "limites_inferiores": limite_inferior, 
    "productos": productos
}

# Para visualización de los datos 

# Vamos a construir un dataframe que contenga Nombre, cantidad_promedio, tasas_nuevos_cliente, demanda_clientes (de productos) 
productos['cantidad_promedio'] = cantidad_promedio.values()
productos['tasas_nuevos_clientes'] = tasas_nuevos_clientes.values()
productos['demanda_clientes'] = productos['demanda_clientes']
productos['limite_inferior'] = productos['limite_inferior']


display(productos)

# Simulación de Colmena 

# Creamos un objeto de tipo simulación
simulacion = Simulacion(query)
# Ejecutamos la simulación
simulacion.run(tiempo_bool = True, tiempo_cero = True)

,Id,Nombre,precio,peso,dimensiones,cantidad,demanda_clientes,productos_solicitados,limite_inferior,cantidad_promedio,tasas_nuevos_clientes
0,0,Frijoles,13,6,6,4652,117,0,11.7,6,4
1,1,Leche,22,12,4,3642,111,0,19.8,3,1
2,2,Cereal,14,10,9,4574,79,0,12.6,2,1
3,3,Arroz,27,13,7,4564,141,0,24.3,6,5
4,4,Fruta,28,5,1,4266,81,0,25.2,4,1
5,5,Jugos,11,11,7,4724,118,0,9.9,5,3


Métodos de descuento aplicados: {'Cupón': 52, 'VideoJuego': 41, 'Trivia': 60, 'Sorteo': 44}


Ingresos totales: 25940.767685246054 y por producto: {'Frijoles': 4071.484470763372, 'Leche': 3094.3088052229655, 'Cereal': 962.9224413180949, 'Arroz': 10511.89630918435, 'Fruta': 4355.028957001589, 'Jugos': 2945.126701755683}


Espacio total: 8691 y peso total: 14492


,Id,Nombre,precio,peso,dimensiones,cantidad,demanda_clientes,productos_solicitados,limite_inferior,cantidad_promedio,tasas_nuevos_clientes
0,0,Frijoles,13,6,6,4652,117,345,11.7,6,4
1,1,Leche,22,12,4,3642,111,160,19.8,3,1
2,2,Cereal,14,10,9,4574,79,79,12.6,2,1
3,3,Arroz,27,13,7,4564,141,429,24.3,6,5
4,4,Fruta,28,5,1,4266,81,167,25.2,4,1
5,5,Jugos,11,11,7,4724,118,300,9.9,5,3


,Id,Nombre,precio,peso,dimensiones,cantidad,demanda_clientes,productos_solicitados,limite_inferior,cantidad_promedio,tasas_nuevos_clientes
0,0,Frijoles,11.653826,6,6,4652,117,1231,11.7,6,4
1,1,Leche,20.532108,12,4,3642,111,227,19.8,3,1
2,2,Cereal,12.948568,10,9,4574,79,120,12.6,2,1
3,3,Arroz,24.179506,13,7,4564,141,1509,24.3,6,5
4,4,Fruta,25.536514,5,1,4266,81,306,25.2,4,1
5,5,Jugos,9.891348,11,7,4724,118,874,9.9,5,3
